# Mini Projekt 4
## Janek Filipecki, Wojtek Zarzecki
### Instrukcje

Zadanie polega na stworzeniu modelu, który będzie klasyfikował liczbę gwiazdek przyznanych hotelowi (rating, klasa = liczba_gwiazdek - 1) na podstawie recenzji, jaką otrzymał (review).
Wszelkie chwyty dozwolone :) Mogą Państwo przewidywać klasy w oparciu o klasyczną metodę bag of words, inne metody reprezentacji dokumentów (np. TF-IDF), korzystać z embeddingów, gotowych modeli językowych itd. Do dyspozycji mają Państwo dane treningowe, oczekuję od Państwa pliku csv z wygenerowanymi predykcjami.
Proszę  zwrócić uwagę na fakt, że jest to problem klasyfikacji wieloklasowej z mocno niezbalansowanym zbiorem danych!
Proszę także Państwa o przetestowanie kilku (w domyśle, co najmniej dwóch) modeli do klasyfikacji - moga (ale nie muszą!) być to gotowe modele, najlepiej o różnych architekturach.

Bardzo proszę, żeby zwrócili mi Państwo archiwum zip (wystarczy jedna osoba z zespołu), proszę też o zastosowanie się do instrukcji:
- Archiwum i wszystkie pliki powinny być nazwane poniedzialek/piatek_nazwisko1_nazwisko2.zip
- W archiwum proszę (bez zbędnych podfolderów!) umieścić pliki ze swoim kodem i testowe predykcje nazwane zgodnie z sekwencją  poniedzialek/piatek_nazwisko1_nazwisko2.csv
- Testowe predykcje powinny mieć kolejność zgodną z kolejnością sekwencji do których się odnoszą w zbiorze testowym. Plik csv nie powinien mieć nagłówka ani indeksów.

Uwaga: proszę dokładnie sprawdzić swoje rozwiązania i predykcje. W związku z końcem semestru, nie będzie możliwości dosyłania poprawek.

In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch import nn
import transformers
import evaluate
import numpy as np
from datasets import *

In [2]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


In [3]:
torch.cuda.empty_cache()

In [4]:
dataset = load_dataset("csv", data_files="data/train_data.csv")
train_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
dataset = DatasetDict({
    'train': train_dataset["train"],
    'test': train_dataset["test"]})
dataset

Found cached dataset csv (C:/Users/filip/.cache/huggingface/datasets/csv/default-756307af69d667af/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at C:\Users\filip\.cache\huggingface\datasets\csv\default-756307af69d667af\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-446f481f583d8575.arrow and C:\Users\filip\.cache\huggingface\datasets\csv\default-756307af69d667af\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-d8858dd4b6606287.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 13113
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 3279
    })
})

In [5]:
len(train_dataset["train"])

13113

In [6]:
train_dataset["train"].features

{'text': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None)}

In [7]:
model_name = "distilbert-base-uncased"

In [8]:
tokenizer = transformers.BertTokenizerFast.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [10]:
tokenized_dataset = train_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\filip\.cache\huggingface\datasets\csv\default-756307af69d667af\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-7171c5e938dff74a.arrow
Loading cached processed dataset at C:\Users\filip\.cache\huggingface\datasets\csv\default-756307af69d667af\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-aba22aebb9a09566.arrow


In [11]:
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
accuracy = evaluate.load("accuracy")

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
model = transformers.BertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForSequenceClassification: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.transformer.layer.5.attention.out_lin.weight', 'distilbert.transformer.layer.3.attention.v_lin.weight', 'distilbert.transformer.layer.5.ffn.lin2.weight', 'distilbert.transformer.layer.4.attention.v_lin.weight', 'distilbert.transformer.layer.4.ffn.lin1.bias', 'distilbert.transformer.layer.2.ffn.lin2.weight', 'distilbert.transformer.layer.4.attention.q_lin.bias', 'distilbert.transformer.layer.3.attention.k_lin.bias', 'distilbert.transformer.layer.5.attention.q_lin.bias', 'distilbert.transformer.layer.1.ffn.lin2.bias', 'distilbert.transformer.layer.3.attention.q_lin.bias', 'distilbert.transformer.layer.4.attention.q_lin.weight', 'distilbert.transformer

In [15]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
training_args = transformers.TrainingArguments(
    output_dir="hotel_rating_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

c:\Users\filip\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1640 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB (GPU 0; 8.00 GiB total capacity; 7.18 GiB already allocated; 0 bytes free; 7.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF